In [1]:
import os

In [2]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [3]:
import sys


def in_kaggle():
    return "kaggle_web_client" in sys.modules


log.info(f"in_kaggle: {in_kaggle()}")

2023-03-20 05:04:46,772 [INFO] [365646948] in_kaggle: False


In [4]:
##### INSERT SOURCE CODE HERE FOR SUBMISSION #####

# PLACE HOLDER

In [5]:
if in_kaggle():
    !cp -r /kaggle/input/graphnet-and-dependencies/software .
    !pip install -q /kaggle/working/software/dependencies/torch-1.11.0+cu115-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl
    !pip install -q /kaggle/working/software/dependencies/torch_geometric-2.2.0.tar.gz
    !cd software/graphnet; pip install -q --no-index --find-links="/kaggle/working/software/dependencies" -e .[torch]

    !cp -r /kaggle/input/hydra-core /kaggle/working
    !mv /kaggle/working/hydra-core/antlr4-python3-runtime-4.9.3.tar.gz.tmp /kaggle/working/hydra-core/antlr4-python3-runtime-4.9.3.tar.gz
    !pip install -q /kaggle/working/hydra-core/*

else:
    %cd ..

/home/jovyan/working


In [6]:
!python 11-inference-parquet.py settings.in_kaggle={in_kaggle()}

graphnet: INFO     2023-03-20 05:04:49 - get_logger - Writing log to logs/graphnet_20230320-050449.log
/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE
  warn(f"Failed to load image Python extension: {e}")
2023-03-20 05:04:50.635515: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 05:04:50.776439: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the

In [7]:
!python 15-validation.py settings.in_kaggle={in_kaggle()}

2023-03-20 05:16:22,885 [INFO][utils] Output directory: output/2023-03-20_05-16-22
2023-03-20 05:16:22,885 [INFO][utils] Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
2023-03-20 05:16:22,885 [INFO][utils] PyTorch version: 1.11.0+cu115
2023-03-20 05:16:22,886 [INFO][utils] Fix seed: 440
2023-03-20 05:16:23,259 [INFO][15-validation] Base score: 1.015247133330943
2023-03-20 05:16:23,400 [INFO][15-validation] Num of low sigma events: 52447, Num of high sigma events: 147553
2023-03-20 05:16:23,400 [INFO][15-validation] Low sigma score: 0.21284382494007598, High sigma score: 1.3004575082691396
2023-03-20 05:16:23,400 [INFO][15-validation] Done.
